## Complexity of the direct convolution method

The number of multiplications performed when calling `convolve(..., method="direct")` doesn't have a pretty formula. I've written up a python function that returns it:

In [5]:
def predicted_muls(S_1, S_2, mode="full"):
    """Prediction of number of multiplications for these shapes and mode"""
    import numpy as np
    if mode == "full":
        if len(S_1) == 1:
            return S_1[0] * S_2[0]
        else:
            return min(np.prod(S_1), np.prod(S_2)) * np.prod([n + k - 1 for n, k in zip(S_1, S_2)])
    elif mode == "valid":
        if len(S_1) == 1:
            S_1, S_2 = S_1[0], S_2[0]
            if S_2 < S_1:
                S_1, S_2 = S_2, S_1
            return (S_2 - S_1 + 1) * S_1
        else:
            return min(np.prod(S_1), np.prod(S_2)) * np.prod([max(n, k) - min(n, k) + 1 for n, k in zip(S_1, S_2)])
    elif mode == "same":
        if len(S_1) == 1:
            S_1, S_2 = S_1[0], S_2[0]
            if S_1 < S_2:
                return S_1 * S_2
            else:
                return S_1 * S_2 - (S_2 // 2) * ((S_2 + 1) // 2)
        else:
            return np.prod(S_1) * np.prod(S_2)

I also claim that the number of additions performed is always less than the number of multiplications performed.

Let's verify this for a few random shapes by counting multiplications and additions when actually running `convolve`.

In [6]:
import numpy as np

class foo:
    """A class that only counts multiplications and additions, and can be used in `convolve`"""
    muls = 0
    adds = 0
    def __mul__(self, other):
        foo.muls += 1
        return self
    def __rmul__(self, other):
        foo.muls += 1
        return self
    def __add__(self, other):
        foo.adds +=1
        return self
    def __radd__(self, other):
        foo.adds +=1
        return self
    def conjugate(self):
        return self
    @staticmethod
    def reset():
        foo.muls = 0
        foo.adds = 0
        
def count_direct_muls_and_adds(S_1, S_2, mode="full"):
    """Count number of multiplications and additions for these shapes and mode"""
    from scipy.signal import convolve
    import numpy as np
    # Reset the counters
    foo.reset()
    # Perform the convolution
    convolve(
        np.array([foo()] * np.prod(S_1)).reshape(S_1),
        np.array([foo()] * np.prod(S_2)).reshape(S_2),
        mode=mode
    )
    # Return the counters
    return foo.muls, foo.adds
    
def test_hypotheses(S_1, S_2, mode="full"):
    """Test the prediction for these shapes and mode"""
    muls_1, adds = count_direct_muls_and_adds(S_1, S_2, mode=mode)
    muls_2 = predicted_muls(S_1, S_2, mode=mode)

    assert muls_1 == muls_2, (S_1, S_2, muls_1, muls_2)
    assert adds <= muls_1, (S_1, S_2, adds, muls_1)
    
# Generate some random data and test the hypotheses
# Go over a few dimenions
for ndim in range(1, 4):
    # Generate random shapes of same dimension
    for S_1, S_2 in np.random.randint(1, 10, (5, 2, ndim)):
        # Test the hypothesis for these shapes
        test_hypotheses(S_1, S_2, mode="full")
        
for ndim in range(1, 4):
    for S_1 in np.random.randint(1, 10, (5, ndim)):
        # In "valid" mode, one shape must be greater than or equal to every dimension of the other shape
        S_2 = S_1 + np.random.randint(0, 3, ndim)
        test_hypotheses(S_1, S_2, mode="valid")
        
for ndim in range(1, 4):
    # Generate random shapes of same dimension
    for S_1, S_2 in np.random.randint(1, 10, (5, 2, ndim)):
        # Test the hypothesis for these shapes
        test_hypotheses(S_1, S_2, mode="same")